In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
import polars as pl
import requests

/Users/ncantalupa/opt/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [75]:
from datetime import datetime

In [10]:
predictions_url = 'https://api-v3.mbta.com/prediction'
with open('api_key.txt') as file:
    api_key = file.read()

In [11]:
routes = requests.get('https://api-v3.mbta.com/routes').json()
for route in routes['data']:
    route_id = route['id']
    print(f'Stop ID: {route_id}')

Stop ID: Red
Stop ID: Mattapan
Stop ID: Orange
Stop ID: Green-B
Stop ID: Green-C
Stop ID: Green-D
Stop ID: Green-E
Stop ID: Blue
Stop ID: 741
Stop ID: 742
Stop ID: 743
Stop ID: 751
Stop ID: 749
Stop ID: 746
Stop ID: CR-Fairmount
Stop ID: CR-Fitchburg
Stop ID: CR-Worcester
Stop ID: CR-Franklin
Stop ID: CR-Greenbush
Stop ID: CR-Haverhill
Stop ID: CR-Kingston
Stop ID: CR-Lowell
Stop ID: CR-Middleborough
Stop ID: CR-Needham
Stop ID: CR-Newburyport
Stop ID: CR-Providence
Stop ID: CR-Foxboro
Stop ID: Boat-F4
Stop ID: Boat-F1
Stop ID: Boat-EastBoston
Stop ID: Boat-Lynn
Stop ID: Boat-F6
Stop ID: 747
Stop ID: 708
Stop ID: 1
Stop ID: 4
Stop ID: 7
Stop ID: 8
Stop ID: 9
Stop ID: 10
Stop ID: 11
Stop ID: 14
Stop ID: 15
Stop ID: 16
Stop ID: 17
Stop ID: 18
Stop ID: 19
Stop ID: 21
Stop ID: 22
Stop ID: 23
Stop ID: 24
Stop ID: 26
Stop ID: 28
Stop ID: 29
Stop ID: 30
Stop ID: 31
Stop ID: 32
Stop ID: 33
Stop ID: 34
Stop ID: 34E
Stop ID: 35
Stop ID: 36
Stop ID: 37
Stop ID: 38
Stop ID: 39
Stop ID: 40
Stop ID:

In [12]:
stops = requests.get('https://api-v3.mbta.com/stops', params={'filter[route]': 'Green-D'}).json()
for stop in stops['data']:
    stop_id = stop['id']
    stop_name = stop['attributes']['name']
    print(f'Stop ID: {stop_id}, Stop Name: {stop_name}')

Stop ID: place-river, Stop Name: Riverside
Stop ID: place-woodl, Stop Name: Woodland
Stop ID: place-waban, Stop Name: Waban
Stop ID: place-eliot, Stop Name: Eliot
Stop ID: place-newtn, Stop Name: Newton Highlands
Stop ID: place-newto, Stop Name: Newton Centre
Stop ID: place-chhil, Stop Name: Chestnut Hill
Stop ID: place-rsmnl, Stop Name: Reservoir
Stop ID: place-bcnfd, Stop Name: Beaconsfield
Stop ID: place-brkhl, Stop Name: Brookline Hills
Stop ID: place-bvmnl, Stop Name: Brookline Village
Stop ID: place-longw, Stop Name: Longwood
Stop ID: place-fenwy, Stop Name: Fenway
Stop ID: place-kencl, Stop Name: Kenmore
Stop ID: place-hymnl, Stop Name: Hynes Convention Center
Stop ID: place-coecl, Stop Name: Copley
Stop ID: place-armnl, Stop Name: Arlington
Stop ID: place-boyls, Stop Name: Boylston
Stop ID: place-pktrm, Stop Name: Park Street
Stop ID: place-gover, Stop Name: Government Center
Stop ID: place-haecl, Stop Name: Haymarket
Stop ID: place-north, Stop Name: North Station
Stop ID: plac

In [13]:
params1 = {'api_key': api_key,
          'filter[stop]': 'place-gover'}

In [17]:
response1 = requests.get(f'https://api-v3.mbta.com/predictions?filter[stop]=place-gover&api_key={api_key}').json()

In [20]:
response1['data'][0]

{'attributes': {'arrival_time': None,
  'arrival_uncertainty': None,
  'departure_time': '2024-05-23T14:02:10-04:00',
  'departure_uncertainty': 360,
  'direction_id': 0,
  'last_trip': False,
  'revenue': 'REVENUE',
  'schedule_relationship': None,
  'status': None,
  'stop_sequence': 40},
 'id': 'prediction-61805336-70202-40',
 'relationships': {'route': {'data': {'id': 'Green-B', 'type': 'route'}},
  'stop': {'data': {'id': '70202', 'type': 'stop'}},
  'trip': {'data': {'id': '61805336', 'type': 'trip'}},
  'vehicle': {'data': {'id': 'G-10143', 'type': 'vehicle'}}},
 'type': 'prediction'}

In [51]:
def make_csv(category):
    response = requests.get(f'https://api-v3.mbta.com/{category}?api_key={api_key}').json()
    data_dict = {}
    for i in response['data']:
        data_dict[i['id']] = i['attributes']
    pd.DataFrame.from_dict(data_dict, orient='index').to_csv(f"{category}.csv")

In [52]:
make_csv("routes")

In [53]:
make_csv("stops")

In [54]:
make_csv("vehicles")

In [55]:
make_csv("lines")

In [80]:
def check_stop(stop_id):
    response = requests.get(f'https://api-v3.mbta.com/predictions?filter[stop]={stop_id}&sort=arrival_time&api_key={api_key}').json()
    for instance in response['data']:
        vehicle_id = instance['relationships']['vehicle']['data']['id']
        arrival_time = instance['attributes']['arrival_time']
        arrival_time = datetime.fromisoformat(arrival_time).strftime("%I:%M %p")
        departure_time = instance['attributes']['departure_time']
        departure_time = datetime.fromisoformat(departure_time).strftime("%I:%M %p")
        direction = instance['attributes']['direction_id']
        route_id = instance['relationships']['route']['data']['id']
        trip_id = instance['relationships']['trip']['data']['id']
        print(f"vehicle: {vehicle_id} || arrival: {arrival_time} || departure: {departure_time} || direction: {direction} || route: {route_id} || trip: {trip_id}")

In [82]:
check_stop('place-hymnl')

vehicle: G-10286 || arrival: 03:33 PM || departure: 03:34 PM || direction: 1 || route: Green-D || trip: 61805106
vehicle: G-10096 || arrival: 03:33 PM || departure: 03:34 PM || direction: 0 || route: Green-C || trip: 61804273
vehicle: G-10034 || arrival: 03:34 PM || departure: 03:35 PM || direction: 0 || route: Green-D || trip: 61805034
vehicle: G-10109 || arrival: 03:36 PM || departure: 03:37 PM || direction: 1 || route: Green-B || trip: ADDED-1581915176
vehicle: G-10285 || arrival: 03:36 PM || departure: 03:37 PM || direction: 1 || route: Green-C || trip: 61804279
vehicle: G-10274 || arrival: 03:39 PM || departure: 03:40 PM || direction: 1 || route: Green-D || trip: 61804902
vehicle: G-10012 || arrival: 03:39 PM || departure: 03:40 PM || direction: 0 || route: Green-B || trip: ADDED-1581915199
vehicle: G-10266 || arrival: 03:41 PM || departure: 03:42 PM || direction: 1 || route: Green-C || trip: ADDED-1581915188
vehicle: G-10231 || arrival: 03:42 PM || departure: 03:43 PM || directio

In [73]:
trip = requests.get('https://api-v3.mbta.com/trips/61804974').json()

In [74]:
trip

{'data': {'attributes': {'bikes_allowed': 2,
   'block_id': 'B800-35',
   'direction_id': 0,
   'headsign': 'Riverside',
   'name': '',
   'revenue': 'REVENUE',
   'wheelchair_accessible': 1},
  'id': '61804974',
  'links': {'self': '/trips/61804974'},
  'relationships': {'route': {'data': {'id': 'Green-D', 'type': 'route'}},
   'route_pattern': {'data': {'id': 'Green-D-855-0', 'type': 'route_pattern'}},
   'service': {'data': {'id': 'SpringWeekday', 'type': 'service'}},
   'shape': {'data': {'id': '8000008', 'type': 'shape'}}},
  'type': 'trip'},
 'jsonapi': {'version': '1.0'}}